# Analyzing the Birthplaces of NHL Players and Their Closest NHL Team

# Imports 

In [ ]:
import pandas as pd
from functions import *
%run functions.py
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from haversine import haversine, Unit # I could have used geopy.distance, too
import scipy.stats as st
import numpy as np 
import pylab 
import mapclassify as mc
import geoplot as gplt
import geoplot.crs as gcrs
import matplotlib.pyplot as plt
import fiona

## Data

In [ ]:
all_players_df = pd.read_csv('data/player_info_df.csv', index_col=0)
all_na_players_df = pd.read_csv('data/all_players_full_info.csv', index_col=0)

In [ ]:
player_gdf = pd.read_csv("data/player_gdf.csv", index_col=0)
player_gdf.drop(columns = ['link','Lat', 'Lng', 'geometry', 'nearest_geom', 'line'], inplace=True)

In [ ]:
player_gdf.head(2)

# Team Analysis

## How Many Players COULD Play for Closest Team

In [ ]:
team_df = pd.DataFrame(player_gdf.groupby('nearest_team').id.count())

### Full teams

In [ ]:
# distribution
sns.violinplot(x=team_df["id"])

In [ ]:
# 11 teams would still be around!
# all 7 canadian teams plus one border team (Detroit, not Buffalo though)
team_df[team_df['id'] > 18].sort_values(by='id', ascending=False)

In [ ]:
print(f"{round(all_na_players_df.shape[0] / all_players_df.shape[0], 2)*100}% of players are from NA")

In [ ]:
# A team roster has 18 players, normally
team_roster_count = player_gdf.shape[0] / player_gdf.team_name.nunique()

In [ ]:
# Since 70% of players are from NA, how many teams have 70% of a roster? 13
## how many teams have more than 13 players born close to them? Same amount as before
team_df[team_df['id'] > 13].sort_values(by='id', ascending=False)

#### Are reasonably close

In [ ]:
team_df_close = pd.DataFrame(player_gdf[player_gdf['close_to_any_team']==True].groupby('nearest_team').id.count())

In [ ]:
# 8 teams have enough players born close to the team
team_df_close[team_df_close['id'] > 13].sort_values(by='id', ascending=False)

## Few Players

In [ ]:
# lots of teh southern teams have 5 or fewer players
# Every team has at least one NHL-quality player near them. That's frankly surprising
team_df[team_df['id'] < 13].sort_values(by='id')

# Players Playing for Home Team

In [ ]:
home_team = player_gdf[player_gdf['team_name'] == player_gdf['nearest_team']]
# only 32 players play for their local team
print(home_team.shape[0])

In [ ]:
# the Bruins and the Maples Leafs have the most hometown players
home_team_groupby = pd.DataFrame(home_team.groupby('team_name').id.count())
home_team_groupby.sort_values(by='id', ascending=False)

## Which team has the highest proportion of local players on their team?

In [ ]:
# since the Maple Leafs have the most local players, it makes sense that they would have the most local players
## which team has the highest proportion of their local players on their team?
### divide values in home_team_groupby by values in closest team
teams_w_local_players = list(home_team_groupby.index)
team_df_w_local_players = team_df[team_df.index.isin(teams_w_local_players)]

In [ ]:
home_team_groupby['Total_local_players_in_NHL'] = team_df_w_local_players
percent_of_local_players = round(home_team_groupby['id'] / 
                                 home_team_groupby['Total_local_players_in_NHL'], 2) * 100
home_team_groupby['Percent_of_local_players_on_team'] = percent_of_local_players
home_team_groupby.sort_values(by='Percent_of_local_players_on_team', ascending=False)

# How many players have their closest team being Seattle? 

In [ ]:
# Seattle will have a new team in 2021
# the new arena's coordinates '47.622, -122.354'

# getting all players who have their closest team being in northwest
players_from_nw = player_gdf[player_gdf['nearest_team'].isin([
                                                             'Vancouver Canucks'])].reset_index(drop=True)

In [ ]:
dist_to_seattle = [haversine([float(players_from_nw['coordinates'][idx].split(",")[0]), 
                              float(players_from_nw['coordinates'][idx].split(",")[1])],
                              [47.622, -122.354],unit=Unit.MILES) for idx in range(len(players_from_nw))]
close_to_seattle = [x for x in dist_to_seattle if x < 60]
# two players were born within 60 miles of seattle 
len(close_to_seattle)

# How many players are REASONABLY close and PLAYING for their NHL team?

In [ ]:
# I'll define reasonably close as 60 miles away (as the crow flies)
## 394 players are very close to A team
very_close_players_df = player_gdf[player_gdf['haversine_distance'] < 60]
very_close_players_df.shape[0]

In [ ]:
# 188 players are not reasonably close to any NHL team (in addition to the 30% of non-American players)
all_na_players_df.shape[0] - very_close_players_df.shape[0]

In [ ]:
# how many players are close to THEIR team? 24
very_close_local_players_df = player_gdf[(player_gdf['haversine_distance'] < 60) & 
                                        (player_gdf['team_name'] == player_gdf['nearest_team'])]
very_close_local_players_df.shape[0]

In [ ]:
# which teams have the most local players?
very_close_groupby = pd.DataFrame(very_close_local_players_df.groupby('team_name').id.count())
very_close_groupby.sort_values(by='id', ascending=False)

# Which States/Provinces Have the Most Players Per Capita?

## Engineering the Population Data

### US

In [ ]:
# had to read files like this b/c it is xlsx file (https://stackoverflow.com/questions/65254535/xlrd-biffh-xlrderror-excel-xlsx-file-not-supported/65255334)
# I got the data from https://www.census.gov/data/datasets/time-series/demo/popest/2010s-state-total.html
us_pop_df = pd.read_excel(
    "data/nst-est2019-01.xlsx",
     engine='openpyxl', skiprows=3
)
us_pop_df2019 = us_pop_df[['Unnamed: 0', 2019]]
us_pop_df2019.columns = ['State_w_dot', 'Population'] # for 2019
us_pop_df2019.drop(index= us_pop_df2019.index[56:], inplace=True)
us_pop_df2019.drop(index= us_pop_df2019.index[:5], inplace=True)
us_pop_df2019['stateProvince'] = [x[1:] for x in us_pop_df2019['State_w_dot']]
us_pop_df2019['Country'] = 'USA'

In [ ]:
# adding State Abbreviations
# from https://worldpopulationreview.com/states/state-abbreviations
us_abbrev = pd.read_csv("data/us_state_abbreviations.csv")

In [ ]:
# adding state abbreviations to the data 
us_pop_df2019_abbrev = pd.merge(us_pop_df2019, us_abbrev, left_on='stateProvince', right_on='State')

### Canada

In [ ]:
can_pop_df = pd.read_csv("data/1710000901-eng.csv", skiprows=5)
can_pop_df.drop(index= can_pop_df.index[15:], inplace=True)
can_pop_df.drop(index= can_pop_df.index[:2], inplace=True)
can_pop_df_2021 = can_pop_df[['Geography', 'Q1 2021']]
can_pop_df_2021.columns = ['Geography', 'Population']
can_pop_df_2021['Population'] = can_pop_df_2021['Population'].str.replace(',', '').astype(float)
can_pop_df_2021['stateProvince'] = [x[:-8] if "(map)" in x else x for x in can_pop_df_2021['Geography']]
can_pop_df_2021['Country'] = 'CAN'

In [ ]:
# canadian abbreviations
can_abbrev = pd.read_excel("data/CountryCAN_e.xls")
can_abbrev['Province'] = [x.title() for x in can_abbrev['Province or Territory Name']]

In [ ]:
# merging the two
can_pop_df_2021_abbrev = pd.merge(can_pop_df_2021, can_abbrev, left_on='stateProvince', right_on='Province')

### Merging the US and Canada Population data

In [ ]:
us_can_pop_df_full = pd.concat([us_pop_df2019_abbrev, can_pop_df_2021_abbrev])

In [ ]:
us_can_pop_df = us_can_pop_df_full[['Population', 'stateProvince', 'Code', 'Country']]

## Combining with the Player Data

In [ ]:
player_province_data = pd.DataFrame(player_gdf.groupby('birthStateProvince').id.count())

In [ ]:
us_can_player_pop_df = pd.merge(us_can_pop_df, player_province_data, 
                                left_on=['Code'], right_on=['birthStateProvince'])

In [ ]:
us_can_player_pop_df.columns = ['Population', 'stateProvince', 'Code', 'Country', 'NHL_players']

In [ ]:
# standardizing how many players there are per 100k inhabitants
us_can_player_pop_df['Players_100k_people'] = round((us_can_player_pop_df['NHL_players'] / us_can_player_pop_df['Population']) * 100000, 2) 

## Analyzing Players Per Capita

In [ ]:
# keeping only the 41 states and provinces with players
us_can_player_pop_df.sort_values(by='Players_100k_people', ascending=False, inplace=True)
us_can_player_pop_df.shape

In [ ]:
us_can_player_pop_df[us_can_player_pop_df['Country']=='USA']

In [ ]:
# plotting all states. Big disparity here!
sns.scatterplot(x = us_can_player_pop_df['Players_100k_people'], y = us_can_player_pop_df['Code'])

In [ ]:
# Canadian provinces
us_can_player_pop_df[us_can_player_pop_df['Country'] == 'CAN']

In [ ]:
# Plotting the top 20 states/provinces
top20_us_can_player_pop_df = us_can_player_pop_df.iloc[0:20].reset_index(drop=True)

In [ ]:
top20_us_can_player_pop_df.head()

In [ ]:
# The average Saskatchewanian is 17 times more likely to play in the nhl than the average New Yorker
top20_us_can_player_pop_df['Players_100k_people'][0] / top20_us_can_player_pop_df['Players_100k_people'][17]

In [ ]:
# viewing the top 20 states/provinces
ax = sns.scatterplot(x = top20_us_can_player_pop_df['Players_100k_people'], y = top20_us_can_player_pop_df['Code'])
ax.set_title("Number of NHL Players per 100,000 People in US and Canadian States and Provinces")

## Analyzing the distribution of NHL Players per Capita

In [ ]:
# all of the players per capita data
get_best_distribution(us_can_player_pop_df['Players_100k_people'])

In [ ]:
# the top 20 players per capita states/provinces
get_best_distribution(top20_us_can_player_pop_df['Players_100k_people'])

### Plotting the distributions

In [ ]:
st.probplot(top20_us_can_player_pop_df['Players_100k_people'], dist=st.lognorm(0.69), plot=pylab)
pylab.show()

# Mapping states by Players per capita

In [ ]:
n_am_gdf = gpd.read_file("data/Political_Boundaries_(Area)/Political_Boundaries_(Area).shp")

In [ ]:
us_can_gdf = n_am_gdf[n_am_gdf['COUNTRY'].isin(['CAN', 'USA'])]
us_can_gdf_dissolved = us_can_gdf.dissolve('NAME')
us_can_gdf_dissolved2 = us_can_gdf_dissolved[us_can_gdf_dissolved.STATEABB.notnull()]

us_can_gdf_dissolved2['ABB'] = [x[-2:] for x in us_can_gdf_dissolved2['STATEABB']]

us_can_gdf_dissolved3 = pd.merge(us_can_gdf_dissolved2, us_can_player_pop_df, left_on='ABB', right_on='Code')

## Choropleth

In [ ]:
us_can_gdf_dissolved3.plot(column = 'NHL_players', figsize=(10, 10), legend=True,
                  legend_kwds={'label':"Total Number of NHL Players"})
# plt.savefig("pictures/province_state_choropleth.png", bbox_inches='tight', pad_inches=0.2)

In [ ]:
us_can_gdf_dissolved3.plot(column = 'Players_100k_people', figsize=(10, 10), legend=True,
                  legend_kwds={'label':"Total Number of NHL Players"})

## Cartogram

In [ ]:
# had to change the projection of it to work
us_can_gdf_dissolved3.to_crs("EPSG:4326", inplace=True)

In [ ]:
gplt.cartogram(us_can_gdf_dissolved3, scale='Players_100k_people', 
               projection=gcrs.AlbersEqualArea(), figsize=(7,7))
plt.title("Basic Cartogram of NA States/Provinces NHL Player Ratio", fontdict={"fontsize":15})

In [ ]:
ax1 = gplt.cartogram(us_can_gdf_dissolved3, scale='Players_100k_people', projection=gcrs.AlbersEqualArea(), 
                     figsize=(15,15), limits=(0.15, 0.95), color = 'green')
gplt.polyplot(us_can_gdf_dissolved3, facecolor='white', edgecolor='grey', ax=ax1)
ax1.set_title("Cartogram of North American States/Provinces NHL Player Ratio", 
              fontdict={"fontsize":15, 'fontweight':'bold'})
plt.savefig("pictures/province_state_cartogram.png")

# Saving Data 

In [ ]:
# smaller version of gdf
us_can_gdf_dissolved4 = us_can_gdf_dissolved3[['geometry', 'ABB', 'Code', 
                                               'Country', 'NHL_players', 'Players_100k_people']]

In [ ]:
us_can_gdf_dissolved4

In [ ]:
# us_can_gdf_dissolved3
us_can_gdf_dissolved4.to_file("data/us_can_geometry.geojson", driver='GeoJSON')